# 1. as_numpy_iterator
- 생성된 Dataset 클래스를 넘파이 배열로 반환
- 텐서로 출력되므로 넘파이 배열로 변환해야 요소 출력 가능

In [1]:
import tensorflow as tf

dataset = tf.data.Dataset.range(10)

list(dataset.as_numpy_iterator())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# 2. apply
- 사용자가 정의한 변환 함수를 Dataset 클래스에 적용

In [2]:
dataset = tf.data.Dataset.range(10)

def filter_five(x):
    return x.filter(lambda x: x < 5)

list(dataset.apply(filter_five).as_numpy_iterator())

[0, 1, 2, 3, 4]

# 3. from_tensor_slices
- 리스트나 넘파이 배열을 Dataset 클래스로 변환


In [3]:
ds = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5])
print(type(ds))
list(ds.as_numpy_iterator())

<class 'tensorflow.python.data.ops.dataset_ops.TensorSliceDataset'>


[1, 2, 3, 4, 5]

# 4. iteration
- 반복문으로 요소를 출력 가능
- 텐서로 출력

In [4]:
ds = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5])

for d in ds:
    print(d)

tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)


# 5. range
- 데이터셋의 요소 생성
- 파이썬의 `range` 내장함수와 동일하게 동작

In [6]:
ds = tf.data.Dataset.range(1, 10, 2)

for d in ds:
    print(d)

tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)


# 6. batch
- 배치 크기 만큼의 요소를 하나의 배치로 구성하여 반환

In [7]:
ds = tf.data.Dataset.range(8)
for d in ds.batch(3):
    print(d)

tf.Tensor([0 1 2], shape=(3,), dtype=int64)
tf.Tensor([3 4 5], shape=(3,), dtype=int64)
tf.Tensor([6 7], shape=(2,), dtype=int64)


# 7. drop_remainder
- 배치 구성 시 잔여 요소의 개수가 배치 크기보다 작다면 생성하지 않음

In [8]:
ds = tf.data.Dataset.range(8)

for d in ds.batch(3, drop_remainder=True):
    print(d)

tf.Tensor([0 1 2], shape=(3,), dtype=int64)
tf.Tensor([3 4 5], shape=(3,), dtype=int64)


# 8. window
- 시계열 데이터셋 구축시 윈도우 구현에 사용
- 윈도우 크기 만큼의 데이터를 하나로 묶어서 생성
- `shift`: 다음 윈도우 구성의 시작점을 몇 칸씩 건너뛸 것인지 정의
- `drop_remainder=False`: 잔여 요소가 배치 크기보다 작더라도 지속 생성

In [10]:
ds = tf.data.Dataset.range(10)
ds = ds.window(5, shift=1, drop_remainder=False)
for d in ds:
    print(list(d.as_numpy_iterator()))

[0, 1, 2, 3, 4]
[1, 2, 3, 4, 5]
[2, 3, 4, 5, 6]
[3, 4, 5, 6, 7]
[4, 5, 6, 7, 8]
[5, 6, 7, 8, 9]
[6, 7, 8, 9]
[7, 8, 9]
[8, 9]
[9]


- 딥러닝 모델에 데이터셋 주입시 크기가 일정하게 맞춰있지 않다면 에러가 발생할 수 있음
    - `drop_remainder=True`로 설정하는 것이 일반

In [11]:
ds = tf.data.Dataset.range(10)
ds = ds.window(5, shift=1, drop_remainder=True)
for d in ds:
    print(list(d.as_numpy_iterator()))

[0, 1, 2, 3, 4]
[1, 2, 3, 4, 5]
[2, 3, 4, 5, 6]
[3, 4, 5, 6, 7]
[4, 5, 6, 7, 8]
[5, 6, 7, 8, 9]


- `shift=2`로 설정 시 2칸씩 건너뛰어 생

In [12]:
ds = tf.data.Dataset.range(10)
ds = ds.window(5, shift=2, drop_remainder=True)
for d in ds:
    print(list(d.as_numpy_iterator()))

[0, 1, 2, 3, 4]
[2, 3, 4, 5, 6]
[4, 5, 6, 7, 8]


# 9. flat_map
- `map`함수를 Dataset 클래스에 적용하며 flatten한 결과 반환

In [13]:
ds = tf.data.Dataset.range(10)
ds = ds.window(5, shift=1, drop_remainder=True)
for d in ds:
    print(d)

<_VariantDataset shapes: (), types: tf.int64>
<_VariantDataset shapes: (), types: tf.int64>
<_VariantDataset shapes: (), types: tf.int64>
<_VariantDataset shapes: (), types: tf.int64>
<_VariantDataset shapes: (), types: tf.int64>
<_VariantDataset shapes: (), types: tf.int64>


- `flat_map` 적용하지 않은 케이스

In [14]:
ds = tf.data.Dataset.range(10)
ds = ds.window(5, shift=1, drop_remainder=True)
ds = ds.flat_map(lambda w: w.batch(5))
for d in ds:
    print(d)

tf.Tensor([0 1 2 3 4], shape=(5,), dtype=int64)
tf.Tensor([1 2 3 4 5], shape=(5,), dtype=int64)
tf.Tensor([2 3 4 5 6], shape=(5,), dtype=int64)
tf.Tensor([3 4 5 6 7], shape=(5,), dtype=int64)
tf.Tensor([4 5 6 7 8], shape=(5,), dtype=int64)
tf.Tensor([5 6 7 8 9], shape=(5,), dtype=int64)


- `flat_map` 적용한 케이스

# 10. shuffle
- 데이터셋을 무작위로 섞음


In [16]:
# shuffle 하지 않은 경우
import numpy as np
ds = tf.data.Dataset.from_tensor_slices(np.arange(10))
for d in ds:
    print(d)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)


In [17]:
# shuffle 한 경우
ds = tf.data.Dataset.from_tensor_slices(np.arange(10)).shuffle(buffer_size=5)
for d in ds:
    print(d)

tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)


# 11.map
- `map` 함수를 Dataset에 적용
- 데이터셋을 입력 데이터와 레이블 데이터로 분리할 때 활용 가능

In [18]:
window_size = 5

ds = tf.data.Dataset.range(10)
ds = ds.window(window_size, shift=1, drop_remainder=True)
ds = ds.flat_map(lambda w: w.batch(window_size))
ds = ds.shuffle(10)

ds = ds.map(lambda x:(x[:-2], x[-2:]))
for x, y in ds:
    print('train set: {}'.format(x))
    print('label set: {}'.format(y))
    print('===' * 10)

train set: [0 1 2]
label set: [3 4]
train set: [4 5 6]
label set: [7 8]
train set: [1 2 3]
label set: [4 5]
train set: [3 4 5]
label set: [6 7]
train set: [2 3 4]
label set: [5 6]
train set: [5 6 7]
label set: [8 9]
